# Tutorial 6: More Informative Rendering

In this tutorial, we will discuss how to make rendering more informative. We will also demonstrate that rendering is agnostic to the selected physics-engine.

The following will be covered:
- Create an overlay node that augments a raw image sensors
- Connect the overlay node and use it for rendering
- Demonstrate that rendering is agnostic to the selected physics-engine

In the remainder of this tutorial, we will go more into detail on this concept.

Furthermore, at the end of this notebook you will find an exercise.
For the exercise you will have to add/modify a couple of lines of code, which are marked by

```python

# START EXERCISE [BLOCK_NUMBER]

# END EXERCISE [BLOCK_NUMBER]
```

## Pendulum Swing-up

We will assume that we already have the object definition of the underactuated pendulum that we used in the [first](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/1_environment_creation.ipynb) tutorial with its dynamics simulated with the [OdeEngine](https://github.com/eager-dev/eagerx_ode). We will also assume that the *engine-specific* implementation we created in the [previous tutorial](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/5_engine_implementation.ipynb) for the [GymEngine](https://github.com/eager-dev/eagerx/blob/master/eagerx/engines/openai_gym/engine.py) is available.

Our goal is to make the rendered images more informative. We will lay the actions, selected by the agent, over the raw images produced by the image sensor of the pendulum. We will then visualise the augmented images instead of the raw images from the image sensor.

## Activate GPU (Colab only)

When in Colab, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package.

In [2]:
try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials >> /tmp/eagerx_install.txt 2>&1"}
    
    # Reload PIL (only required the first time)
    import importlib
    import PIL
    importlib.reload(PIL.TiffTags)

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()

# Import eagerx
import eagerx
eagerx.set_log_level(eagerx.WARN)

Not running on CoLab.


## Let's get started

We will again create an environment with the *Pendulum* object, like we did in the [first](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/1_environment_creation.ipynb) and [second](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/2_reset_and_step.ipynb) tutorials.

Let's make the *Pendulum* object and add it to an empty graph.

In [3]:
# Make the pendulum
from eagerx_tutorials.pendulum.objects import Pendulum
import eagerx_tutorials.pendulum.gym_implementation  # noqa: registers gym implementation
pendulum = Pendulum.make("pendulum", actuators=["u"], sensors=["theta", "dtheta", "image"], states=["model_state"])

# Define rate in Hz
rate = 30.0

# Initialize empty graph
graph = eagerx.Graph.create()

# Add pendulum to the graph
graph.add(pendulum)

# Connect the pendulum to an action and observation
graph.connect(action="voltage", target=pendulum.actuators.u)
graph.connect(source=pendulum.sensors.theta, observation="angle")
graph.connect(source=pendulum.sensors.dtheta, observation="angular_velocity")

Below we have defined a overlay node. The callback receives two inputs, namely the applied action `u` and a `raw_image`. In the callback, the applied action is visualised as an overlay on top of the raw image. The resulting `image` is the output of the callback. If we render this output, instead of the raw `pendulum.sensors.image`, we get a more informative rendered image.

In the exercise of this tutorial, we will finish the code that defines the Overlay node below. Specifically, we will overlay some text on top of the image that indicates the current timestamp.

Similar to what we covered in [this](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/4_nodes.ipynb) tutorial, we can create this node by inheriting from the class [`Node`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html).
This class has the following abstract methods we need to implement:

- [`make()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.make): Makes the parameter specification of the node.
- [`initialize()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.initialize): Initializes the node.
- [`reset()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.reset): Resets the node at the beginning of an episode.
- [`callback()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.callback): Called at the rate of the node.


In [4]:
from eagerx import register, Space
from eagerx.core.specs import NodeSpec
from eagerx.utils.utils import Msg
import cv2
import numpy as np


class Overlay(eagerx.Node):
    @classmethod
    def make(
        cls,
        name: str,
        rate: float,
        process: int = eagerx.ENVIRONMENT,
        color: str = "cyan",
    ) -> NodeSpec:
        """Overlay spec"""
        # Get base parameter specification with defaults parameters
        spec = cls.get_specification()

        # Adjust default params
        spec.config.update(name=name, rate=rate, process=process, color=color)
        spec.config.update(inputs=["base_image", "u", "theta"], outputs=["image"])
        return spec
        
    def initialize(self, spec: NodeSpec):
        """Nothing to initialize"""
        pass

    @register.states()
    def reset(self):
        """Nothing to reset (i.e. stateless node)"""
        pass

    @register.inputs(
        base_image=Space(dtype="uint8"),
        u=Space(low=[-2], high=[2]),
        theta=Space(shape=(), dtype="float32"),
    )
    @register.outputs(image=Space(dtype="uint8"))
    def callback(self, t_n: float, base_image: Msg, u: Msg, theta: Msg):
        if len(base_image.msgs[-1].data) > 0:
            u = u.msgs[-1].data[0] if u else 0
            theta = theta.msgs[-1]

            # Set background image from base_image
            img = base_image.msgs[-1]
            height, width, _ = img.shape
            side_length = min(width, height)

            # Put text
            font = cv2.FONT_HERSHEY_SIMPLEX
            text = "Applied Voltage"
            text_size = cv2.getTextSize(text, font, 0.5, 2)[0]
            text_x = int((width - text_size[0]) / 2)
            text_y = int(text_size[1])
            img = cv2.putText(img, text, (text_x, text_y), font, 0.5, (0, 0, 0))

            # Draw grey bar
            img = cv2.rectangle(
                img,
                (width // 2 - side_length * 4 // 10, height // 2 - side_length * 9 // 20),
                (width // 2 + 4 * side_length // 10, height // 2 - 4 * side_length // 10),
                (125, 125, 125),
                -1,
            )

            # Fill bar proportional to the action that is applied
            p1 = (width // 2, height // 2 - side_length * 9 // 20)
            p2 = (width // 2 + int(side_length * u * 2 / 15), height // 2 - 4 * side_length // 10)
            img = cv2.rectangle(img, p1, p2, (0, 0, 0), -1)

            # START EXERCISE 1.3

            # START EXERCISE 1.3

            # Add theta info
            img = cv2.putText(
                img, f"theta ={theta: .2f} rad", (text_x, height - int(2.2 * text_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0)
            )

            return dict(image=img)
        else:
            return dict(image=np.zeros((0, 0, 3), dtype="uint8"))


Below, we wil initially render the raw images produced by the pendulum image sensor as we did in the preceding tutorials. In the exercise of this tutorial, we will:
- Add the overlay node to the graph.
- Connect the inputs of the overlay node. I.e. `u` to the `voltage` action and `pendulum.sensors.image` to the `overlay.inputs.base_image`.
- Change the render source from `pendulum.sensors.image` to `overlay.outputs.image`.

In [5]:
# Make the overlay node parameter specification
overlay = Overlay.make("overlay", rate)

# Copy the spaces of the actuators and sensors to the overlay node.
overlay.inputs.u.space = pendulum.actuators.u.space
overlay.inputs.base_image.space = pendulum.sensors.image.space
overlay.outputs.image.space = pendulum.sensors.image.space
    
# Add overlay node to graph and connect it
# START EXERCISE 1.1
# Add the overlay node to the graph and connect the following:
# - pendulum.sensors.image to overlay.inputs.base_image 
# - action "voltage" to overlay.inputs.u
# - pendulum.sensors.theta to overlay.inputs.theta

# END EXERCISE 1.1

# Define the render source
# START EXERCISE 1.2
# Change the render source to overlay.outputs.image.
graph.render(source=pendulum.sensors.image, rate=rate)
# START EXERCISE 1.2

Next, we will define the environment.

In [6]:
from typing import Dict
import numpy as np


class PendulumEnv(eagerx.BaseEnv):
    def __init__(self, name: str, rate: float, graph: eagerx.Graph, engine: eagerx.Engine):
        """Initializes an environment with EAGERx dynamics.

        :param name: The name of the environment. Everything related to this environment
                     (parameters, topics, nodes, etc...) will be registered under namespace: "/[name]".
        :param rate: The rate (Hz) at which the environment will run.
        :param graph: The graph consisting of nodes and objects that describe the environment's dynamics.
        :param engine: The physics engine that will govern the environment's dynamics.
        """
        # Make the backend specification
        from eagerx.backends.single_process import SingleProcess
        backend = SingleProcess.make()
        
        self.eval = eval
        
        # Maximum episode length
        self.max_steps = 100
        
        # Step counter
        self.steps = None
        super().__init__(name, rate, graph, engine, backend, force_start=True)
    
    def step(self, action: Dict):
        """A method that runs one timestep of the environment's dynamics.

        :params action: A dictionary of actions provided by the agent.
        :returns: A tuple (observation, reward, done, info).

            - observation: Dictionary of observations of the current timestep.

            - reward: amount of reward returned after previous action

            - done: whether the episode has ended, in which case further step() calls will return undefined results

            - info: contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        # Take step
        observation = self._step(action)
        self.steps += 1
        
        # Get angle and angular velocity
        # Take first element because of window size (covered in other tutorial)
        th = observation["angle"][0]
        thdot = observation["angular_velocity"][0]

        # Convert from numpy array to float
        u = float(action["voltage"])

        # Calculate cost
        # Penalize angle error, angular velocity and input voltage
        cost = th**2 + 0.1 * thdot**2 + 0.001 * u**2  

        # Determine when is the episode over
        # currently just a timeout after 100 steps
        done = self.steps > self.max_steps

        # Set info, tell the algorithm the termination was due to a timeout
        # (the episode was truncated)
        info = {"TimeLimit.truncated": self.steps > self.max_steps}
        
        return observation, -cost, done, info
    
    def reset(self) -> Dict:
        """Resets the environment to an initial state and returns an initial observation.

        :returns: The initial observation.
        """
        # Determine reset states
        states = self.state_space.sample()
            
        # Perform reset
        observation = self._reset(states)

        # Reset step counter
        self.steps = 0
        return observation

Next, we will intially use the OdeEngine, as we did in the preceding tutorials. Later on in the exercises. we will switch and use the [GymEngine](https://github.com/eager-dev/eagerx/blob/master/eagerx/engines/openai_gym/engine.py) instead.

Because the overlay node is agnostic to the *engine-specific* implementation of the pendulum object, it will naturally overlay the additional information on whatever image it receives. Hence, this allows informative rendering to be available, whatever physics-engine is chosen.

In [7]:
# Import the two supported engines
from eagerx_ode.engine import OdeEngine
from eagerx.engines.openai_gym.engine import GymEngine

# Make the engine
# START EXERCISE 1.4
engine = OdeEngine.make(rate=rate)
# engine = GymEngine.make(rate=rate, process=eagerx.ENVIRONMENT)
# END EXERCISE 1.4

At this point, we have create a graph containing the pendulum. We provide the graph to the environment together with the engine. Based on this engine, we will initialize the *engine-specific implementation* for the pendulum that was registered with this engine. 
- If we use the [OdeEngine](https://github.com/eager-dev/eagerx_ode), the raw sensor images are produced by the custom render function in the registered OdeEngine implementation [here](https://github.com/eager-dev/eagerx_tutorials/blob/3ddc2eb7558c7825095611fec3a01a47f5e7af79/eagerx_tutorials/pendulum/objects.py#L108-L168).
- If we use the [GymEngine](https://github.com/eager-dev/eagerx/blob/master/eagerx/engines/openai_gym/engine.py), the raw sensor images are produced by the [Pendulum-v1](https://gym.openai.com/envs/Pendulum-v0/) environment.
- If we would have an implemention for the real-world and registered it with the [RealEngine](https://github.com/eager-dev/eagerx_reality/blob/m1aster/eagerx_reality/engine.py), the raw sensor images could, for example, be produced by a real camera.

Finally, we train the agent using [Stable Baselines3](https://stable-baselines3.readthedocs.io/en/master/), again similar to the preceding tutorials.

In [8]:
import stable_baselines3 as sb3
from stable_baselines3.common.env_checker import check_env
from eagerx.wrappers import Flatten
from gym.wrappers.rescale_action import RescaleAction

# Initialize Environment
env = PendulumEnv(name="PendulumEnv", rate=rate, graph=graph, engine=engine)

# Print action & observation space
print("action_space: ", env.action_space)
print("observation_space: ", env.observation_space)

# Stable Baselines3 expects flattened actions & observations
# Convert observation and action space from Dict() to Box(), normalize actions
env = Flatten(env)
env = RescaleAction(env, min_action=-1.0, max_action=1.0)

# Check that env follows Gym API and returns expected shapes
check_env(env)

# Toggle render
env.render("human")

# Initialize learner
model = sb3.SAC("MlpPolicy", env, verbose=1)

# Train for 1 minute (sim time)
model.learn(total_timesteps=int(60 * rate))

env.shutdown()

[WARN]: Backend 'SINGLE_PROCESS' does not support multiprocessing, so all nodes are launched in the ENVIRONMENT process.
action_space:  Dict(voltage:Space([-2.], [2.], (1,), float32))
observation_space:  Dict(angle:Box([-999.], [999.], (1,), float32), angular_velocity:Box([-999.], [999.], (1,), float32))
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 101       |
|    ep_rew_mean     | -1.32e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 73        |
|    time_elapsed    | 5         |
|    total_timesteps | 404       |
| train/             |           |
|    actor_loss      | 24.1      |
|    critic_loss     | 16.9      |
|    ent_coef        | 0.914     |
|    ent_coef_loss   | -0.14     |
|    learning_rate   | 0.0003    |
|    n_updates       | 303       |
----------------------------------
----

# Exercise

In this exercise you will create a node that overlays the applied actions over raw images that are produced by the image sensor of the pendulum. As the overlay node is agnostic to the physics-engine, we have the same overlay in every physics-engine.

For this exercise, you will need to modify or add some lines of code in the cells above.
These lines are indicated by the following comments:

```python
# START EXERCISE [BLOCK_NUMBER]

# END EXERCISE [BLOCK_NUMBER]
```

However, feel free to play with the other code as well if you are interested.
We recommend you to restart and run all code after each section (in Colab there is the option *Restart and run all* under *Runtime*).

## 1. Render more informative images


### Add your code to the following blocks: 

1.1 Add the overlay node to the graph and connect the inputs `overlay.inputs.raw_image` and `overlay.inputs.u` to `pendulum.sensors.image` and action `voltage`, respectively.  
1.2 Change the render source to `overlay.outputs.image`. Using the [*eagerx_gui* package](https://github.com/eager-dev/eagerx_gui), you would see that the graph looks as below if `graph.gui()` would be called. Run the code, and you should now see the rendered overlay instead of the raw sensor images. 

<img src="./figures/tutorial_6_gui.svg" width=720>

1.3 In the callback of the overlay node, add the current time (i.e. `t_n`) as text to the image. Run the code, and you should see a timestamp that increase while the episode progresses.  
1.4 Select the `GymEngine` by uncommenting the marked line. Run the code, and you should see that the base image has changed, but the overlay is nevertheless put on top. Hence, this demonstrates the agnostic behavior of the `graph`. 